In [10]:
import httpx
import base64
from rasterio.io import MemoryFile
from rasterio.plot import show
import numpy as np
from base64 import b64decode
import rasterio
from rasterio import features

In [16]:
URL = "https://cerulean-cloud-test-cloud-run-orchestrator-5qkjkyomta-ew.a.run.app"
# FILL THIS IN
API_KEY="###"

In [17]:
orchestrator_result = httpx.post(URL+"/orchestrate", json={"sceneid": "S1A_IW_GRDH_1SDV_20200729T034859_20200729T034924_033664_03E6D3_93EF"}, timeout=None, headers={"Authorization": f"Bearer {API_KEY}"})


print(orchestrator_result)
# Takes 10min for 1 scene syncronously
# 1 min 6s with 3 concurrency

In [ ]:
result = orchestrator_result.json()
print(result.keys())

In [ ]:
print(result["classification_base"])
print(result["classification_offset"])

In [ ]:
import geopandas as gpd
import pandas as pd
pd.options.mode.chained_assignment = None

def reproject_to_utm(gdf_wgs84):
    utm_crs = gdf_wgs84.estimate_utm_crs(datum_name='WGS 84')
    return gdf_wgs84.to_crs(utm_crs)

def remap_class_to_priotity(row):
    if row['classification_left'] > 0 and  row['classification_right'] > 0 and row['classification_left'] !=  row['classification_right']:
        return 2
    elif row['classification_left'] > 0 and  row['classification_right'] > 0 and row['classification_left'] ==  row['classification_right']:
        return row['classification_left']
    else:
        return 0
    
def get_intersect_and_disjoints(grid_base, grid_offset, sjoin_result_inner):
    
    grid_offset_disjoint = grid_offset.loc[~grid_offset.index.isin(sjoin_result_inner.index), :]
    grid_base_disjoint = grid_base.loc[~grid_base.index.isin(sjoin_result_inner["index_right"]), :]
    
    # if the offset prediction does not match the base prediction, we divide confidence by two
    # TODO there could be various policies for handling this especially with more than two
    # positive classes and this can be abstracted out
    grid_offset_disjoint.loc[:,"confidence"] /= 2
    grid_base_disjoint.loc[:,"confidence"] /= 2
    
    sjoin_result_inner = sjoin_result_inner.rename(columns={"classification_left": "classification", "confidence_left": "confidence"})
    for column in sjoin_result_inner.columns:
        if column not in grid_base.columns:
            sjoin_result_inner = sjoin_result_inner.drop(column, axis=1)
    return pd.concat([grid_offset_disjoint,grid_base_disjoint, sjoin_result_inner])

In [ ]:
grid_base = gpd.GeoDataFrame.from_features(result["classification_base"])

In [ ]:
grid_offset = gpd.GeoDataFrame.from_features(result["classification_offset"])

In [ ]:
grid_offset = grid_offset.set_crs(4326)
grid_base = grid_base.set_crs(4326)

grid_offset.to_file("offset.geojson")
grid_base.to_file("base.geojson")

In [ ]:
grid_base = reproject_to_utm(grid_base)
grid_offset = reproject_to_utm(grid_offset)

In [ ]:
sjoin_result_inner = grid_offset.sjoin_nearest(grid_base, how="inner", max_distance=500,  distance_col="join_distance")

In [18]:
sjoin_result_inner['classification_left'] = sjoin_result_inner.apply(remap_class_to_priotity, axis=1)
sjoin_result_inner['classification_right'] = sjoin_result_inner.apply(remap_class_to_priotity, axis=1)

In [48]:
merged_grid_inferences = get_intersect_and_disjoints(grid_base, grid_offset, sjoin_result_inner)

In [45]:
merged_grid_inferences.to_file("merged_inferences.geojson")

/home/rave/miniconda3/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


buffering by 70 meter pixel * 8 pixels from a glance that an example inference with grid edge effects had 6 pixels separation

# https://stackoverflow.com/questions/67280722/how-to-merge-touching-polygons-with-geopandas

In [ ]:
merged_grid_inferences_w_buffer = merged_grid_inferences.copy()
merged_grid_inferences_buffered_geoms = merged_grid_inferences.buffer(5*70)

In [76]:
merged_grid_inferences_w_buffer['geometry'] = merged_grid_inferences_buffered_geoms

In [77]:
merged_grid_inferences_w_buffer.dissolve().buffer(-1*70).to_file("merged_buffered_dilated.geojson")


/home/rave/miniconda3/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
